In [ ]:
# Read the PDF
import textract
beers_pdf = textract.process('RealAle2017.pdf')

In [ ]:
# Process raw text 
import string
import re

beers_pdf_lines = string.split(beers_pdf, '\n')

# If line starts with at least two Capital letters -> Brewery
# Example: 'ALLENDALE - Allendale, Northumberland'
# Then, every line with a percentage at the end -> beer
# Example:  'Wagtail Best Bitter - 3.8%'
brewery_pattern = re.compile('^[A-Z]{2}')
beer_pattern = re.compile("%$")

beers_list = []

for line in beers_pdf_lines:

    if brewery_pattern.search(line) and not beer_pattern.search(line):  # re.match() replaced with re.search()
        brewery_name = line.split(' -')[0]
        #print brewery_name
    
    # Replace long dashes by short dashes 
    if beer_pattern.search(line):
        if ('\xe2\x80\x93') in line:
            line = line.replace('\xe2\x80\x93', "-")
        if ('\x0c') in line:
            line = line.replace('\x0c','')

        # Name of the beer before the alcohol percentage
        beer_name = line.split(' -')[0]
        #print beer_name
            
        # Add beer to list as dictionary
        beers_list.append({'brewery': brewery_name, 'name': beer_name})
            
beers_list

In [ ]:
print "Number of beers " + str(len(beers_list))

In [ ]:
# Search in google for the beers by name: Brewery + beer_name
from google import search

beer_counter = 1
for beer in beers_list:
    searchable_name = beer['brewery'] + ' ' + beer['name']
    print '[' + str(beer_counter) + '] Searching for ' + searchable_name
    beer_counter = beer_counter + 1 

    # Search in google for the beer url in RateBeer
    ratebeer_pattern = re.compile("ratebeer.com")
    beer_url = ""
    for url in search(searchable_name, stop=3):
        if ratebeer_pattern.search(url):
            beer_url = url
            break
    
    beer.update({'url' : beer_url})
    
beers_list

In [ ]:
import csv
import sys

# Save the list in a csv file to avoid looking for the urls again
keys = beers_list[0].keys()
print keys

with open('beers_ealing_festival_2017_urls_only.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(beers_list)

In [ ]:
# Read from csv (after manual update to fill some empty values)
with open('beers_ealing_festival_2017_urls_only (updated).csv') as csvfile:
    reader = csv.DictReader(csvfile)
    beers_list = [row for row in reader]

In [ ]:
# Read rates and style from Ratebeer
import requests
from bs4 import BeautifulSoup

#Example url='https://www.ratebeer.com/beer/allendale-pennine-pale/203829/'

beer_counter = 1
for beer in beers_list:
    print '[' + str(beer_counter) + '] Checking beer ' + beer['name'] + ' at ' + beer['url']
    beer_counter = beer_counter + 1
    
    # Initial values
    overall_rate = 0
    style_rate = 0
    style = 'unknown'
    
    if beer['url'] is not '':
        # Dump data from url
        r = requests.get(beer['url'])
        data = r.text

        # Get rates and style
        soup = BeautifulSoup(data)

        overall_rate = soup.find("div", {"class" : "ratingValue"})
        if overall_rate is None:
            print 'overall_rate not found'
        else:
            overall_rate = overall_rate.string

        style_rate = soup.find("div", {"class" : "style-text"})
        if style_rate is None:
            print 'style_rate not found'
        else:
            style_rate = style_rate.previous_sibling.previous_sibling

        style = soup.find("span", {"id" : "styleTopFifty"})
        if style is None:
            print 'style not found'
        else:
            style = style.previous_sibling.string

    beer.update({'overall_rate' : overall_rate})
    beer.update({'style_rate' : style_rate})
    beer.update({'style' : style})

In [ ]:
# Encode certain styles or names
# TODO: Fix the patch to be solid
for beer in beers_list:
    if beer['style'] is not None:
        beer['style'] = beer['style'].encode('utf-8')
        print beer['style']

In [ ]:
import csv
import sys

# Save the list in a csv file
beers_list

keys = beers_list[0].keys()
print keys

with open('beers_ealing_festival_2017.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(beers_list)